In [42]:
import numpy as np
import pandas as pd
import optuna
from sklearn import preprocessing
#import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.manifold import TSNE
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch import optim
#from geomloss import SamplesLoss
from torch.autograd import Function
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.nn.functional import normalize
#from torchmetrics.classification import BinaryAccuracy
#from torchmetrics.classification import BinaryF1Score
from sklearn.linear_model import LogisticRegression
torch.manual_seed(0)

In [43]:
class TarNet(nn.Module):
    def __init__(self,params):
        super(TarNet, self).__init__()
        self.encoder1 = nn.Linear(25, params['RL11'])
        self.encoder2 = nn.Linear(params['RL11'], params['RL21'])
        self.encoder3 = nn.Linear(params['RL21'], params['RL32'])

        self.regressor1_y0 = nn.Sequential(
            nn.Linear(params['RL32'], params['RG012']),
            nn.ELU(),
            nn.Dropout(p=.01),
        )
        self.regressor2_y0 = nn.Sequential(
            nn.Linear(params['RG012'], params['RG022']),
            nn.ELU(),
            nn.Dropout(p=.01),
        )
        self.regressorO_y0 = nn.Linear(params['RG022'], 1)

        self.regressor1_y1 = nn.Sequential(
            nn.Linear(params['RL32'], params['RG112']),
            nn.ELU(),
            nn.Dropout(p=.01),
        )
        self.regressor2_y1 = nn.Sequential(
            nn.Linear(params['RG112'], params['RG122']),
            nn.ELU(),
            nn.Dropout(p=.01),
        )
        self.regressorO_y1 = nn.Linear(params['RG122'], 1)


    def forward(self, inputs):
        x = nn.functional.elu(self.encoder1(inputs))
        x = nn.functional.elu(self.encoder2(x))
        phi = nn.functional.elu(self.encoder3(x))

        out_y0 = self.regressor1_y0(phi)
        out_y0 = self.regressor2_y0(out_y0)
        y0 = self.regressorO_y0(out_y0)

        out_y1 = self.regressor1_y1(phi)
        out_y1 = self.regressor2_y1(out_y1)
        y1 = self.regressorO_y1(out_y1)

        concat = torch.cat((y0, y1), 1)
        return concat,phi

    '''
    class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, 50)
        self.linear_2 = torch.nn.Linear(50, output_dim)
    def forward(self, x):
        out_1=self.linear(x)
        out_2=self.linear_2(out_1)
        outputs = torch.sigmoid(out_2)
        return outputs
    
    '''


In [44]:
def objective(trial,i):

    params = {
          'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-4),
          'optimizer': trial.suggest_categorical("optimizer", ["Adam", "SGD"]),
          'batch_size':trial.suggest_int('batch_size', 8, 256),
          'RL11':trial.suggest_int('RL11', 16, 512),
          'RL21': trial.suggest_int('RL21', 16, 512),
          'RL32': trial.suggest_int('RL32', 16, 512),
          'RG012':trial.suggest_int('RG012', 16, 512),
        'RG022':trial.suggest_int('RG022', 16, 512),
        'RG112':trial.suggest_int('RG112', 16, 512),
        'RG122':trial.suggest_int('RG122', 16, 512),
          
          }

    model = TarNet(params)

    pehe,model= train_evaluate(params, model, trial,i)

    return pehe

In [45]:
class Data(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X.astype(np.float32))
        self.y = torch.from_numpy(y.astype(np.float32))
        self.len = self.X.shape[0]
       
    def __getitem__(self, index):
        return self.X[index], self.y[index]
   
    def __len__(self):
        return self.len

In [46]:
def get_data(data_type,file_num):

    if(data_type=='train'):
        data=pd.read_csv(f"Dataset/IHDP_a/ihdp_npci_train_{file_num}.csv")
    else:
        data = pd.read_csv(f"Dataset/IHDP_a/ihdp_npci_test_{file_num}.csv")

    x_data=pd.concat([data.iloc[:,0], data.iloc[:, 1:30]], axis = 1)
    #x_data=data.iloc[:, 5:30]
    y_data=data.iloc[:, 1]
    return x_data,y_data

In [47]:
def get_dataloader(x_data,y_data,batch_size):

    x_train_sr=x_data[x_data['treatment']==0]
    y_train_sr=y_data[x_data['treatment']==0]
    x_train_tr=x_data[x_data['treatment']==1]
    y_train_tr=y_data[x_data['treatment']==1]


    train_data_sr = Data(np.array(x_train_sr), np.array(y_train_sr))
    train_dataloader_sr = DataLoader(dataset=train_data_sr, batch_size=batch_size)

    train_data_tr = Data(np.array(x_train_tr), np.array(y_train_tr))
    train_dataloader_tr = DataLoader(dataset=train_data_tr, batch_size=batch_size)


    return train_dataloader_sr, train_dataloader_tr

In [48]:
def regression_loss(concat_true, concat_pred):
    #computes a standard MSE loss for TARNet
    y_true = concat_true[:, 0] #get individual vectors
    t_true = concat_true[:, 1]

    y0_pred = concat_pred[:, 0]
    y1_pred = concat_pred[:, 1]

    #Each head outputs a prediction for both potential outcomes
    #We use t_true as a switch to only calculate the factual loss
    loss0 = torch.sum((1. - t_true) * torch.square(y_true - y0_pred))
    loss1 = torch.sum(t_true * torch.square(y_true - y1_pred))
    #note Shi uses tf.reduce_sum for her losses instead of tf.reduce_mean.
    #They should be equivalent but it's possible that having larger gradients accelerates convergence.
    #You can always try changing it!
    return loss0 + loss1

In [49]:
def cal_pehe(data,y,model):
    #data,y=get_data('test',i)

    data=data.to_numpy()
    data=torch.from_numpy(data.astype(np.float32))



    concat_pred,phi=model(data[:,5:30])
    #dont forget to rescale the outcome before estimation!
    #y0_pred = data['y_scaler'].inverse_transform(concat_pred[:, 0].reshape(-1, 1))
    #y1_pred = data['y_scaler'].inverse_transform(concat_pred[:, 1].reshape(-1, 1))
    cate_pred=concat_pred[:,1]-concat_pred[:,0]
    cate_true=data[:,4]-data[:,3] #Hill's noiseless true values


    cate_err=torch.mean( torch.square( ( (cate_true) - (cate_pred) ) ) )

    return torch.sqrt(cate_err).item()


In [50]:
def loss_cal(X_data,y_data,net):
    
    x_train_sr=X_data[X_data['treatment']==0]
    y_train_sr=y_data[X_data['treatment']==0]
    x_train_tr=X_data[X_data['treatment']==1]
    y_train_tr=y_data[X_data['treatment']==1]
    xs_t=x_train_sr.iloc[:,0].to_numpy()
    xt_t=x_train_tr.iloc[:,0].to_numpy()
    
    xs=x_train_sr.iloc[:,5:30].to_numpy()
    xt=x_train_tr.iloc[:,5:30].to_numpy()
    xs_t=torch.from_numpy(xs_t.astype(np.float32))
    xt_t=torch.from_numpy(xt_t.astype(np.float32))
    y_train_sr=y_train_sr.to_numpy()
    y_train_tr=y_train_tr.to_numpy()
    xs=torch.from_numpy(xs.astype(np.float32))
    xt=torch.from_numpy(xt.astype(np.float32))
    
    y_train_sr=torch.from_numpy(y_train_sr.astype(np.float32))
    y_train_tr=torch.from_numpy(y_train_tr.astype(np.float32))
    
    
    input_data=torch.cat((xs,xt),0)
    true_y=torch.unsqueeze(torch.cat((y_train_sr,y_train_tr),0), dim=1)
    true_t=torch.unsqueeze(torch.cat((xs_t,xt_t),0), dim=1)
    
    
    concat_true=torch.cat((true_y,true_t),1)
    concat_pred,phi=net(input_data)
    loss=regression_loss(concat_true, concat_pred)
    
    return loss.item()

    

In [51]:
def train_log_reg(X_train,y_train,phi_model,phi_size):
    epochs = 400
    input_dim = 25 # Two inputs x1 and x2 
    output_dim = 1 # Single binary output 
    learning_rate = 0.1
    model = LogisticRegression(phi_size,output_dim)
    criterion = torch.nn.BCELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    #x_data,y_data=get_data('train',1)
    #X_train, X_val,y_train, y_val = train_test_split(x_data,y_data ,random_state=42, test_size=0.20)
    #batch_loss=0
    train_loss=[]
    #val_loss=[]
    batch_size=32
    for i in range(epochs):
            batch_loss=0

            train_dataloader_sr, train_dataloader_tr=get_dataloader(X_train,y_train,batch_size)

            for batch_idx, (train_source_data, train_target_data) in enumerate(zip(train_dataloader_sr, train_dataloader_tr)):

                xs,ys=train_source_data
                xt,yt=train_target_data

                xs_train=xs[:,5:30]
                xt_train=xt[:,5:30]

                train_x=torch.cat((xs_train,xt_train),0)
                #train_y=torch.unsqueeze(torch.cat((ys,yt),0), dim=1)
                true_t=torch.unsqueeze(torch.cat((xs[:,0],xt[:,0]),0), dim=1)
                #concat_true=torch.cat((train_y,true_t),1)
                concat_pred,phi=phi_model(train_x)

                optimizer.zero_grad()
               # Setting our stored gradients equal to zero
                outputs = model(phi)
                loss = criterion(outputs, true_t) # [200,1] -squeeze-> [200]
                loss.backward() # Computes the gradient of the given tensor w.r.t. graph leaves 
                batch_loss=batch_loss+loss.item()
                optimizer.step() 
            train_loss.append(batch_loss)
            
    #X_train_n=X_train.to_numpy()
    #X_train_n=torch.from_numpy(X_train_n.astype(np.float32))
    #train_probs=model(X_train_n[:,5:30])
    #return train_probs,1-train_probs
    return model
        

In [52]:
y_MSE=nn.MSELoss()
#criterion_reg=nn.MSELoss()
#criterion_reg=regression_loss(concat_true,concat_pred)
epochs=300
#batch_size=32

In [53]:
train_loss=[]
val_loss=[]
pehe_error=[]
batch_loss=0
num_files=2
lambda_=0.3
def train_evaluate(param, model, trial,file_num):
    #for nf in range(1,num_files):
    x_data,y_data=get_data('train',file_num)
    X_train, X_val,y_train, y_val = train_test_split(x_data,y_data ,
                                       random_state=42, 
                                       test_size=0.20)
    
    #net=TarNet(25,.01)
    #opt_net = torch.optim.Adam(net.parameters(), lr=1e-4)
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    
    optimizer = getattr(optim, param['optimizer'])(model.parameters(), lr= param['learning_rate'])
    
   
        #model = model.cuda()
    #model = model
        #criterion = criterion.cuda()

    for ep in range(1,epochs+1 ):
        #print('epoch',ep)
        batch_loss=0
        #X_train_n=X_train.to_numpy()
        #X_train_n=torch.from_numpy(X_train_n.astype(np.float32))
        #concat_pred_b,phi_b=model(X_train_n[:,5:30])
        #logistic_reg=train_log_reg(X_train,y_train,model,param['RL32'])
        
        #------------------------------
        
        t_com=X_train.iloc[:,0].to_numpy()
        x_data_com=X_train.iloc[:,5:30].to_numpy()
        x_data_com = torch.from_numpy(x_data_com.astype(np.float32))
        concat_pred_com,phi_com=model(x_data_com)

        torch.nan_to_num(phi_com,nan=0.0)
        phi_com=phi_com.detach().numpy()
        
        
        clf = LogisticRegression(max_iter=400,random_state=0).fit(phi_com, t_com) #
        #clf = LogisticRegression(random_state=0).fit(trans_comp, t_comp)
        
        #----------------------------
        

        train_dataloader_sr, train_dataloader_tr=get_dataloader(X_train,y_train,param['batch_size'])

        for batch_idx, (train_source_data, train_target_data) in enumerate(zip(train_dataloader_sr, train_dataloader_tr)):

            xs,ys=train_source_data
            xt,yt=train_target_data

            xs_train=xs[:,5:30]
            xt_train=xt[:,5:30]

            train_x=torch.cat((xs_train,xt_train),0)
            train_y=torch.unsqueeze(torch.cat((ys,yt),0), dim=1)
            true_t=torch.unsqueeze(torch.cat((xs[:,0],xt[:,0]),0), dim=1)
            concat_true=torch.cat((train_y,true_t),1)
            concat_pred,phi=model(train_x)
            
            
            # p(t|x)
            
            #t_com=t_batch
            
            #print(phi_com)
            #phi_com[np.isnan(phi_com)] = 0
            t_batch=torch.cat((xs[:,0],xt[:,0]),0).numpy()
            #t_comp=torch.cat((xs[:,0],xt[:,0]),0).detach().numpy()
            trans_comp=phi.detach().numpy()
            p=clf.predict_proba(trans_comp)
            #p_1=logistic_reg(phi)
            #p_0=1-p_1
            #true_prob=[1-t_comp.mean(), t_comp.mean()]
            true_prob=[1-t_batch.mean(), t_batch.mean()]

            #scaler = preprocessing.StandardScaler().fit(phi_com)
            #tranformed=scaler.transform(phi_com)


            model.zero_grad()

            #source_mse=criterion_reg(y0,ys)
            #target_mse=criterion_reg(y1,yt)
            ptx=torch.unsqueeze(torch.from_numpy(p[1].astype(np.float32)), dim=1)
            #ptx=p_1
            pt=torch.empty(ptx.shape[0],1).fill_(true_prob[1])
            
            sel_loss=torch.sum(torch.abs(ptx-pt))
            #May be there is another way
            combined_loss=(lambda_)*regression_loss(concat_true,concat_pred)+(1-lambda_)*sel_loss # think about tradeoff and other
            # loss function, check in paper
            #print('Training loss: ',combined_loss.item())
            # backward propagation
            combined_loss.backward()
            losss=batch_loss+combined_loss.item()

            # optimize
            optimizer.step()
        #train_loss.append(loss_cal(X_train,y_train,net))
        #val_loss.append(loss_cal(X_val,y_val,net))
        train_loss.append(losss)
        
        # Add prune mechanism
        #trial.report(accuracy, ep)

        #if trial.should_prune():
        #   raise optuna.exceptions.TrialPruned()
            
    #return cal_pehe(X_val,y_val,model),model
    return loss_cal(X_val,y_val,model),model

        
        

In [54]:
pehe_total=[]
for i in range(1,2):
    func = lambda trial: objective(trial, i)
    study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=42))
    study.optimize(func, n_trials=10)
    best_trial = study.best_trial
    print('Files completed so far = ',i)
    best_model=TarNet(study.best_trial.params)
    train_loss.clear()
    best_val,model=train_evaluate(study.best_trial.params, best_model, study.best_trial,i)
    data,y=get_data('test',i)
    pehe=cal_pehe(data,y,model)

    pehe_total.append(pehe)


[I 2023-05-19 18:32:56,110] A new study created in memory with name: no-name-c4c9911f-dfdf-4f1d-a730-65d3fed7e59f
/tmp/ipykernel_342389/2216607761.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-4),
[I 2023-05-19 18:33:29,444] Trial 0 finished with value: 1191.3082275390625 and parameters: {'learning_rate': 2.368863950364079e-05, 'optimizer': 'Adam', 'batch_size': 157, 'RL11': 93, 'RL21': 93, 'RL32': 44, 'RG012': 446, 'RG022': 314, 'RG112': 367, 'RG122': 26}. Best is trial 0 with value: 1191.3082275390625.
/tmp/ipykernel_342389/2216607761.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_f

Files completed so far =  1


In [55]:
print(np.mean(pehe_total))

0.7116636037826538


In [ ]:
np.savetxt("v3_Ours_sel_loss_1_50_(IHDPa-Hyper_val_300ep_outsample).csv", pehe_total,delimiter =", ", fmt ='% s')

In [15]:
pehe_total

[0.7473039031028748,
 0.6983595490455627,
 0.9662187099456787,
 0.7418673634529114,
 0.8065203428268433,
 1.1107197999954224,
 0.4231216311454773,
 0.5315221548080444,
 0.8781972527503967,
 1.4119138717651367,
 1.3753641843795776,
 0.5007650256156921,
 0.6108106374740601,
 1.3233189582824707,
 0.8348250389099121,
 0.6897068023681641,
 0.662876546382904,
 0.5743263959884644,
 0.8990809321403503,
 1.0691567659378052,
 0.7262060642242432,
 0.7096218466758728,
 0.6990131139755249,
 0.5855550169944763,
 0.48488813638687134,
 0.9634784460067749,
 0.963718593120575,
 0.7788012027740479,
 1.54386305809021,
 0.6514948010444641,
 0.4629833400249481,
 0.7672489285469055,
 0.6974350810050964,
 0.778927743434906,
 0.6609624624252319,
 0.48041534423828125,
 0.45375707745552063,
 0.48724886775016785,
 1.1568771600723267,
 2.1152191162109375,
 1.7150038480758667,
 0.6617791056632996,
 0.32774782180786133,
 0.9956676959991455,
 0.753425121307373,
 0.7662010192871094,
 0.8731015920639038,
 1.33742964267

In [ ]:
#plt.plot(train_loss)

In [29]:
np.mean(pehe_total)

0.9699601031313039

In [44]:
#for key, value in best_trial.params.items():
#    print("{}: {}".format(key, value))

[]

In [ ]:
#ate_pred=torch.mean(cate_pred)
#print("Estimated ATE (True is 4):", ate_pred.detach().numpy(),'\n\n')

#print("Individualized CATE Estimates: BLUE")
#print(pd.Series(cate_pred.detach().numpy()).plot.kde(color='blue'))
#print("Individualized CATE True: Green")
#print(pd.Series(cate_true.detach().numpy()).plot.kde(color='green'))

#print("\nError CATE Estimates: RED")
#print(pd.Series(cate_pred.detach().numpy()-cate_true.detach().numpy()).plot.kde(color='red'))